In [80]:
import pandas as pd
import numpy as np
import random
import collections

In [10]:
import os
os.chdir('/home/kamil/Downloads/musdata/index')

In [85]:
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "/home/kamil/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/kamil/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/kamil/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/kamil/anaconda3/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/kamil/anaconda3/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime

In [15]:
music = pd.read_csv('3.csv',sep='\t',index_col=0)

In [25]:
music.head()

,pitch,offset
0,117,0.0
1,123,8.0
2,129,12.0
3,94,16.0
4,134,16.0


In [50]:
words_dic = {}
for file in os.listdir():
    music = pd.read_csv(file,sep='\t',index_col=0)
    dic = {}
    for row in music.iterrows():
        offset = row[1]['offset']
        note = row[1]['pitch']
        try:
            dic[offset].append(note)
        except:
            dic[offset] = []
            dic[offset].append(note)
    #convert lists to tuples since they are immutable and can be used as dic keys
    for key, value in dic.items():
        dic[key] = tuple(dic[key])
    #we now create our dictionary of words
    for key, value in dic.items():
        try:
            words_dic[value] = words_dic[value] + 1
        except:
            words_dic[value] = 1

In [66]:
music = pd.read_csv('3.csv',sep='\t',index_col=0)
dic = {}
for row in music.iterrows():
    offset = row[1]['offset']
    note = row[1]['pitch']
    try:
        dic[offset].append(note)
    except:
        dic[offset] = []
        dic[offset].append(note)
for key, value in dic.items():
    dic[key] = tuple(dic[key])

In [69]:
words = list(dic.values())

In [45]:
#convert lists to tuples since they are immutable and can be used as dic keys
#for key, value in dic.items():
#    dic[key] = tuple(dic[key])

In [46]:
#words_dic = {}
#for key, value in dic.items():
#    try:
#        words_dic[value] = words_dic[value] + 1
#    except:
#        words_dic[value] = 1

[((129.0,), 19579),
 ((123.0,), 18509),
 ((112.0,), 17059),
 ((106.0,), 16071),
 ((135.0,), 15351),
 ((146.0,), 14809),
 ((140.0,), 14390),
 ((118.0,), 12341),
 ((152.0,), 11575),
 ((100.0,), 11494)]

In [60]:
vocabulary_size = len(words_dic)

In [61]:
count = [['UNK', -1]]
count.extend(collections.Counter(words_dic).most_common(vocabulary_size - 1))

In [ ]:
count

In [70]:
# BUILD DATASET
#count = [['UNK', -1]]
#count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
dictionary = dict()
for word, _ in count:
    dictionary[word] = len(dictionary)
data = list()
unk_count = 0
for word in words:
    if word in dictionary:
        index = dictionary[word]
    else:
        index = 0  # dictionary['UNK']
        unk_count += 1
    data.append(index)
count[0][1] = unk_count
reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
#return data, count, dictionary, reverse_dictionary

In [78]:
data_index = 0

In [74]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [81]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)

In [83]:
labels

array([[ 546],
       [1066],
       [ 146],
       [   4],
       [7607],
       [ 546],
       [   2],
       [   4]], dtype=int32)

In [17]:
music['offset'].head()

0     0.0
1     8.0
2    12.0
3    16.0
4    16.0
Name: offset, dtype: float64